<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FLIGHTPLANAGENT_OPENAI_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai -q
!pip install colab-env -q
import colab_env
!pip install python-dotenv -q

In [ ]:
import openai
import pandas as pd
from sklearn.linear_model import LogisticRegression
import os
import re
import json

# Set OpenAI API key
openai.api_key = os.environ.get("OPENAI_API_KEY")

class FlightEnvironment:
    def __init__(self, origin, destination, weather):
        self.origin = origin
        self.destination = destination
        self.weather = weather

    def get_possible_routes(self):
        routes = {
            ("Montreal", "London"): [
                {
                    "flight_number": "AC888",
                    "departure_time": "20:00",
                    "arrival_time": "08:00",
                    "price": 800,
                    "airline": "Air Canada",
                    "aircraft_type": "Boeing 787-9"
                },
                {
                    "flight_number": "UA901",
                    "departure_time": "22:00",
                    "arrival_time": "10:00",
                    "price": 900,
                    "airline": "United Airlines",
                    "aircraft_type": "Boeing 777-300ER"
                }
            ]
        }
        return routes.get((self.origin, self.destination), [])

    def get_weather_data(self, location):
        return self.weather.get(
            location, {"weather": [{"description": "Sunny"}], "wind": {"speed": 5}}
        )

class FlightAgent:
    def __init__(self, environment, turbulence_dataset_path):
        self.environment = environment
        self.turbulence_model = LogisticRegression()
        self.turbulence_dataset_path = turbulence_dataset_path
        self.train_turbulence_model()

    def train_turbulence_model(self):
        turbulence_df = pd.read_csv(self.turbulence_dataset_path)
        turbulence_df['Turbulence'] = (turbulence_df['REF_k'] > turbulence_df['REF_k'].mean()).astype(int)
        target = turbulence_df['Turbulence']
        features = turbulence_df[
            [
                "REF_U_1",
                "REF_U_2",
                "REF_U_3",
                "REF_tau_11",
                "REF_tau_12",
                "REF_tau_13",
                "REF_tau_22",
                "REF_tau_23",
                "REF_tau_33",
            ]
        ]
        self.turbulence_model.fit(features, target)
        print("Turbulence model trained successfully!")

    def generate_flight_plan(self):
        origin_weather = self.environment.get_weather_data(self.environment.origin)
        destination_weather = self.environment.get_weather_data(self.environment.destination)
        possible_routes = self.environment.get_possible_routes()

        # Your original enhanced prompt
        prompt = f"""
        Generate a detailed flight plan for a long-haul flight from {self.environment.origin} to {self.environment.destination},
        considering the following:

        ## Recommended Flight and Aircraft Type:
        - Consider these available flights: {possible_routes}
        - Only consider flights operated by Air Canada (AC) or United Airlines (UA).
        - Based on the available flights, recommend the most suitable flight and provide a justification for your choice.
        - **Clearly state the aircraft type for the recommended flight (e.g., Boeing 777, Airbus A330).**

        ## Route and Waypoints:
        - Determine and include a list of suitable waypoints for the recommended flight, using standard aviation codes (e.g., KJFK, EGLL).
        - Consider typical North Atlantic Tracks (NATs) for optimal routing and fuel efficiency.
        - The origin and destination airport codes are: {self.environment.origin} and {self.environment.destination}.

        ## Fuel Calculations:
        - **Provide a detailed breakdown of fuel requirements for different phases of the flight (e.g., taxi, takeoff, climb, cruise, descent, landing).**
        - Include the total fuel required for the trip.

        ## ETOPS Considerations:
        - State the ETOPS certification for the recommended aircraft.
        - List suitable ETOPS alternate airports.

        ## Risks and Mitigation Strategies:
        - **Provide detailed explanations of potential risks associated with this specific flight and aircraft type, along with specific mitigation strategies.**
        - Consider risks such as crew fatigue, jet lag, clear air turbulence, decompression, and medical emergencies.

        ## Additional Information for Pilot and Crew:
        - Provide any relevant information for the pilot and crew, such as NOTAMs, airspace restrictions, oceanic procedures, and potential delays.
        """

        # Initialize the conversation history
        messages = [{"role": "system", "content": prompt}]

        # Get the response from the OpenAI Responses API
        response = openai.chat.completions.create(
            #model="gpt-4-1106-preview",
            model="gpt-4o",
            messages=messages,
        )

        flight_plan = response.choices[0].message.content

        # Post-processing
        flight_plan = re.sub(r'\n+', '\n', flight_plan)
        flight_plan = re.sub(r'## (.*)', r'\n\n**\1**:', flight_plan)

        return flight_plan

# Initialize environment and agent
environment = FlightEnvironment(
    origin="Montreal",
    destination="London",
    weather={
        "Montreal": {"weather": [{"description": "Clear"}], "wind": {"speed": 5}},
        "London": {"weather": [{"description": "Overcast"}], "wind": {"speed": 10}}
    }
)

agent = FlightAgent(environment, '/content/gdrive/MyDrive/datasets/turbulence/REF.csv')

# Generate and print the flight plan
flight_plan = agent.generate_flight_plan()
print('\n\nFlight plan:\n', flight_plan)